# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [144]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import *
from IPython.core.display import HTML

#### Creating list of filepaths to process original event csv data files

In [145]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    #print(root)
    if '.ipynb_checkpoints' not in root:
        file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [146]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    #print(f)
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        # Note: newline='' is always recommended as new lines inside quoted
        # fields are ignored otherwise.
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("No. of rows:", len(full_data_rows_list)) #: 8056
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            # These rows don't have an artist name (they correspond to events which are not NEXT_SONG)
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


No. of rows: 8056


In [147]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("No. of rows in CSV File: ", sum(1 for line in f))

No. of rows in CSV File:  6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [148]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [149]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class': 'SimpleStrategy', 'replication_factor': 1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [150]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


#### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

#### Create queries to ask the following three questions of the data

##### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


##### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

##### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'






#### Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Create required table *get_songmetadata_from_sessioninfo*:
Create a table called '*get_songmetadata_from_sessioninfo*' which corresponds to this query. The primary key which can be used to uniquely identify a row is (sessionid, iteminsession). Here, the partition key is sessionid and the results are ordered by the clustering column iteminsession. As the query wants us to return the artist, song and song length (i.e., the song metadata, which explains the name of the table), we include those columns in the table as well.

In [151]:
create_query1 = "CREATE TABLE IF NOT EXISTS get_songmetadata_from_sessioninfo "
create_query1 = create_query1 + "(sessionid int, iteminsession int, artist text, song text, length decimal, PRIMARY KEY (sessionid, iteminsession))"

try:
    session.execute(create_query1)
except Exception as e:
    print(e)

### Insert data from event_datafile_new.csv into get_songmetadata_from_sessioninfo.

In [153]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for artist, firstname, gender, iteminsession, lastname, length,\
                level, location, sessionid, song, userid in csvreader:
        query = "INSERT INTO get_songmetadata_from_sessioninfo (sessionid, iteminsession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        # Change data types as the Cassandra driver doesn't seem to do automatic type conversion
        try: 
            session.execute(query, (int(sessionid), int(iteminsession), artist, song, Decimal(length)))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data has been inserted: test directly by running the required query (without using SELECT *). 
Show the results in a Pandas DataFrame. The query asks us to display only 3 columns in the result, so the 2 columns in the  where condition (sessionid and iteminsession) are not displayed. The results correspond to sessionid = 338 AND iteminsession = 4.

In [155]:
select_query1 = "SELECT artist, song, length FROM get_songmetadata_from_sessioninfo WHERE sessionid = 338 AND iteminsession = 4" 
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
display_columns = ['Artist', 'Song', 'Song Length']
df = pd.DataFrame(columns=display_columns)
for row in rows:
    # Note: the row object converts field names into lowercase
    df_temp = pd.DataFrame([[row.artist, row.song, row.length]],
                           columns=display_columns)
    df = df.append(df_temp)
    
display(HTML(df.to_html(index=False)))

Artist,Song,Song Length
Faithless,Music Matters (Mark Knight Dub),495.3073


### Query 2. Give me only the following: name of artist, song (sorted by iteminsession) and user (first and last name) for userid = 10, sessionid = 182

#### Create required table *get_song_and_user_metadata_from_usersessions*:
Create a table called '*get_song_and_user_metadata_from_usersessions*' which corresponds to this query. The primary key which can be used to uniquely identify a row is (userid, sessionid, iteminsession). Here, the partition key is userid and the results are ordered by the 2 clustering columns, sessionid and iteminsession. As the query requires the artist, song and user details (first name and last name), we include those columns in the table as well.

In [156]:
create_query2 = "CREATE TABLE IF NOT EXISTS get_song_and_user_metadata_from_usersessions "
create_query2 = create_query2 + "(userid int, sessionid int, iteminsession int, artist text, song text, firstname text, lastname text, PRIMARY KEY (userid, sessionid, iteminsession))"

try:
    session.execute(create_query2)
except Exception as e:
    print(e)

### Insert data from event_datafile_new.csv into get_song_and_user_metadata_from_usersessions.

In [157]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for artist, firstname, gender, iteminsession, lastname, length,\
                level, location, sessionid, song, userid in csvreader:
        query = "INSERT INTO get_song_and_user_metadata_from_usersessions (userid, sessionid, iteminsession, artist, song, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Change data types as the Cassandra driver doesn't seem to do automatic type conversion
        try: 
            session.execute(query, (int(userid), int(sessionid), int(iteminsession), artist, song, firstname, lastname))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data has been inserted: test directly by running the required query (without using SELECT *). 
Show the results in a Pandas DataFrame. The query asks us to display 4 columns in the result along with the sorting column iteminsession, so the 2 columns in the where condition (userid and sessionid) are excluded. The results are already ordered by iteminsession for the given sessionid, as iteminsession is the second clustering key. There is therefore no need for an explicit ORDER BY statement.  

In [162]:
select_query2 = "SELECT artist, song, firstname, lastname, iteminsession FROM get_song_and_user_metadata_from_usersessions WHERE userid = 10 AND sessionid = 182" 
try:
    rows = session.execute(select_query2)
except Exception as e:
    print(e)
display_columns = ["Artist", "Song", "User's First Name", "User's Last Name", 'Item In Session']
df = pd.DataFrame(columns=display_columns)
for row in rows:
    # Note: the row object converts field names into lowercase
    df_temp = pd.DataFrame([[row.artist, row.song, row.firstname, row.lastname, row.iteminsession]],
                           columns=display_columns)
    df = df.append(df_temp)
    
display(HTML(df.to_html(index=False)))

Artist,Song,User's First Name,User's Last Name,Item In Session
Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
Three Drives,Greece 2000,Sylvie,Cruz,1
Sebastien Tellier,Kilometer,Sylvie,Cruz,2
Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


                    

#### Create required table *get_users_who_listened_to_song*:
Create a table called '*get_users_who_listened_to_song*' which corresponds to this query. The primary key which can be used to uniquely identify a row is the (song, userid). We use the userid as part of the primary key rather than the first name and the last name because the user id is guaranteed to be unique for every user. Also, for this query, we don't really care if the same user (same userid) listened to the same song multiple times. So even if there are multiple rows with the same (song, userid) combination, but with different (sessionid, iteminsession) values, only one of these rows really matters to us. 

The partition key is the song title, and the clustering column is the userid. As the query wants us to return the first and last names, we include those columns in the table as well.

In [163]:
create_query3 = "CREATE TABLE IF NOT EXISTS get_users_who_listened_to_song "
create_query3 = create_query3 + "(song text, userid int, firstname text, lastname text, PRIMARY KEY (song, userid))"

try:
    session.execute(create_query3)
except Exception as e:
    print(e)

### Insert data from event_datafile_new.csv into get_users_who_listened_to_song.

In [164]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for artist, firstname, gender, iteminsession, lastname, length,\
                level, location, sessionid, song, userid in csvreader:
        query = "INSERT INTO get_users_who_listened_to_song (song, userid, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Change data types as the Cassandra driver doesn't seem to do automatic type conversion
        try: 
            session.execute(query, (song, int(userid), firstname, lastname))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted: test directly by running the required query (without using SELECT *). 
Show the results in a Pandas DataFrame. We show the user's first and last name along with the song title (which can be excluded, but has been included for clarity) in the results. The results are sorted by userid, which is itself not displayed.

In [166]:
select_query1 = "SELECT song, firstname, lastname FROM get_users_who_listened_to_song WHERE song = 'All Hands Against His Own'" 
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
display_columns = ["Song", "User's First Name", "User's Last Name"]
df = pd.DataFrame(columns=display_columns)
for row in rows:
    # Note: the row object converts field names into lowercase
    df_temp = pd.DataFrame([[row.song, row.firstname, row.lastname]],
                           columns=display_columns)
    df = df.append(df_temp)
    
display(HTML(df.to_html(index=False)))

Song,User's First Name,User's Last Name
All Hands Against His Own,Jacqueline,Lynch
All Hands Against His Own,Tegan,Levine
All Hands Against His Own,Sara,Johnson


### Drop the tables before closing out the sessions

In [167]:
drop_query1 = "DROP TABLE IF EXISTS get_songmetadata_from_sessioninfo "
try:
    rows = session.execute(drop_query1)
except Exception as e:
    print(e)

drop_query2 = "DROP TABLE IF EXISTS get_song_and_user_metadata_from_usersessions "
try:
    rows = session.execute(drop_query2)
except Exception as e:
    print(e)
    
drop_query3 = "DROP TABLE IF EXISTS get_users_who_listened_to_song "
try:
    rows = session.execute(drop_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [168]:
session.shutdown()
cluster.shutdown()